Import du csv et traitement

In [1]:
import pandas as pd

# Spécifie le chemin vers ton fichier CSV
chemin_fichier_csv = 'https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz'

# Définis la taille des chunks (nombre de lignes par chunk)
chunksize = 10000  # Par exemple, 10000 lignes par chunk
listedechunk = []

# Colonnes sélectionnées
selected_cols = [
    "code",
    "product_name",
    "created_datetime",
    "quantity",
    "brands",
    "categories",
    'categories_en',
    'pnns_groups_1',
    'pnns_groups_2',
    'main_category_en',
    "ingredients_text",
    "nutriscore_score",
    "nutriscore_grade",
    "countries_en",
    "energy-kj_100g",
    "energy-kcal_100g",
    "fat_100g",
    "saturated-fat_100g",
    "omega-3-fat_100g",
    "omega-6-fat_100g",
    "sugars_100g",
    "added-sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "fruits-vegetables-nuts-estimate-from-ingredients_100g",
    "nutrition-score-fr_100g"
]

# Parcourt le fichier CSV en utilisant les chunks
for i, chunk in enumerate(pd.read_csv(chemin_fichier_csv, sep="\t", chunksize=chunksize, compression='gzip', on_bad_lines="skip", low_memory=False)):
    # Sélectionne les colonnes nécessaires dans chaque chunk
    chunk = chunk[selected_cols]
   # Convertir la colonne 'created_datetime' en datetime
    chunk.loc[:, 'created_datetime'] = pd.to_datetime(chunk['created_datetime'], errors='coerce')

    # Ajoute le chunk traité à la liste
    listedechunk.append(chunk)



Traitement du csv

In [2]:
# Parcourt les chunks déjà stockés dans listedechunk
for i in range(len(listedechunk)):
    chunk = listedechunk[i]  # Récupère le chunk actuel

    # Supprime les lignes où nutriscore_grade est 'unknown' ou 'not-applicable'
    chunk = chunk[chunk['nutriscore_grade'].isin(['a', 'b', 'c', 'd', 'e'])]
    # Remplace le chunk original par le chunk nettoyé
    listedechunk[i] = chunk


Test sur un chunk

In [3]:
if listedechunk:
    premier_chunk = listedechunk[0]
    premier_chunk.to_csv('premier_chunk.csv', index=False)  # Enregistre sans les index
    print("Le premier chunk a été enregistré dans 'premier_chunk.csv'.")
else:
    print("Aucun chunk n'a été chargé.")

Le premier chunk a été enregistré dans 'premier_chunk.csv'.


In [4]:
# Concatène tous les chunks en un seul DataFrame
df_concatene = pd.concat(listedechunk)

#vire les colonnes vide de 90%
threshold= len(df_concatene)*0.1
df_concatene = df_concatene.dropna(thresh=threshold, axis=1)




Faire avec les pays

In [ ]:

import pycountry

# Crée un ensemble de pays valides
valid_countries = set(country.name for country in pycountry.countries)


def filter_valid_countries(countries_str):
    if pd.isnull(countries_str):  # Vérifie si la chaîne est None
        return ['unknown']
    
    countries = countries_str.split(',')
    return [country for country in countries if country in valid_countries]

# Appliquer la fonction sur la colonne countries_en
df_concatene['countries_en'] = df_concatene['countries_en'].apply(filter_valid_countries)

# Remplir les lignes vides par 'unknown'
df_concatene['countries_en'] = df_concatene['countries_en'].apply(lambda x: x if x else ['unknown'])

# Sauvegarde le DataFrame final dans un fichier CSV
df_concatene.to_csv('fichier_concatene_avecpays.csv', index=False)


On fait des plot

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Supposons que df_concatene a été créé précédemment

# 1. Convertir 'created_datetime' en datetime
df_concatene['created_datetime'] = pd.to_datetime(df_concatene['created_datetime'], errors='coerce')

# 2. Extraire l'année seulement
df_concatene['created_year'] = df_concatene['created_datetime'].dt.year
df_concatene = df_concatene.dropna(subset=['created_year'])  # Supprime les lignes sans année

# Vérifie les colonnes et les types de données
print(df_concatene.columns)
print(df_concatene['created_datetime'].dtype)

# 3. Aplatir la colonne 'countries_en'
df_concatene_exploded = df_concatene.explode('countries_en')

# 4. Graphique de création des produits par année (bar graph)
plt.figure(figsize=(10, 6))
sns.countplot(data=df_concatene, x='created_year')
plt.title('Nombre de Produits Créés par Année')
plt.xlabel('Année')
plt.ylabel('Nombre de Produits')
plt.xticks(rotation=45)

# Ajustements supplémentaires
plt.tight_layout()
plt.subplots_adjust(bottom=0.2, top=0.9)  # Ajuste les marges inférieure et supérieure

plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# 3. Aplatir la colonne 'countries_en'
df_concatene_exploded = df_concatene.explode('countries_en')



# 5. Tableau de répartition par pays
country_distribution = df_concatene_exploded['countries_en'].value_counts()

# Affichage du tableau
print("Répartition des Produits par Pays:")
print(country_distribution)

# Optionnel: Afficher un graphique de la répartition par pays
plt.figure(figsize=(10, 6))
country_distribution.plot(kind='bar', color='skyblue')
plt.title('Répartition des Produits par Pays')
plt.xlabel('Pays')
plt.ylabel('Nombre de Produits')
plt.xticks(rotation=90)
plt.tight_layout()
plt.subplots_adjust(bottom=0.2, top=0.9)  # Ajuste les marges inférieure et supérieure

plt.show()


plot pnaas grpoup etc


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# 4. Aplatir les catégories si nécessaire

df_concatene_exploded = df_concatene.explode('pnns_groups_1')



# 7. Tableau de répartition des catégories
category_distribution = df_concatene_exploded['pnns_groups_1'].value_counts()

print("Répartition des Produits par pnns_groups_1:")
print(category_distribution)
# Optionnel: Afficher un graphique de la répartition par catégories
plt.figure(figsize=(10, 6))
category_distribution.plot(kind='bar', color='lightgreen')
plt.title('Répartition des Produits par Catégories')
plt.xlabel('Catégories')
plt.ylabel('Nombre de Produits')
plt.xticks(rotation=90)
plt.tight_layout()
plt.subplots_adjust(bottom=0.2, top=0.9)  # Ajuste les marges inférieure et supérieure
plt.show()


In [ ]:

df_concatene_exploded = df_concatene_exploded.explode('pnns_groups_2')

# 7. Tableau de répartition des catégories
category_distribution = df_concatene_exploded['pnns_groups_2'].value_counts()

print("Répartition des Produits par pnns_groups_2:")
print(category_distribution)
# Optionnel: Afficher un graphique de la répartition par catégories
plt.figure(figsize=(10, 6))
category_distribution.plot(kind='bar', color='lightgreen')
plt.title('Répartition des Produits par Catégories')
plt.xlabel('Catégories')
plt.ylabel('Nombre de Produits')
plt.xticks(rotation=90)
plt.tight_layout()
plt.subplots_adjust(bottom=0.2, top=0.9)  # Ajuste les marges inférieure et supérieure
plt.show()




Graph demandés


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# 1. Extraire uniquement l'année de la date de création
df_concatene_exploded['created_datetime'] = pd.to_datetime(df_concatene_exploded['created_datetime'], errors='coerce')
df_concatene_exploded['created_year'] = df_concatene_exploded['created_datetime'].dt.year

# Supprimer les lignes sans année
df_concatene_exploded = df_concatene_exploded.dropna(subset=['created_year'])

# Créer un bargraph pour les créations de produits par année
plt.figure(figsize=(10, 6))
sns.countplot(data=df_concatene_exploded, x='created_year', color='skyblue')
plt.title('Nombre de Produits Créés par Année')
plt.xlabel('Année')
plt.ylabel('Nombre de Produits')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# 3. Aplatir la colonne 'countries_en'
df_concatene_exploded = df_concatene.explode('countries_en')
# 5. Tableau de répartition par pays
country_distribution = df_concatene_exploded['countries_en'].value_counts()

# Affichage du tableau
print("Répartition des Produits par Pays:")
print(country_distribution)

# Optionnel: Afficher un graphique de la répartition par pays
plt.figure(figsize=(10, 6))
country_distribution.plot(kind='bar', color='skyblue')
plt.title('Répartition des Produits par Pays')
plt.xlabel('Pays')
plt.ylabel('Nombre de Produits')
plt.xticks(rotation=90)
plt.tight_layout()
plt.subplots_adjust(bottom=0.2, top=0.9)  # Ajuste les marges inférieure et supérieure

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Convertir toutes les valeurs en minuscules et supprimer les espaces en trop
df_concatene_exploded['pnns_groups_2'] = df_concatene_exploded['pnns_groups_2'].str.lower().str.strip()

# Exploser les listes en lignes individuelles
df_pnns_exploded = df_concatene_exploded.explode('pnns_groups_2')

# Supprimer les espaces en trop après l'explosion
df_pnns_exploded['pnns_groups_2'] = df_pnns_exploded['pnns_groups_2'].str.strip()

# Supprimer les apostrophes (ou autres caractères indésirables) en fin de mot
df_pnns_exploded['pnns_groups_2'] = df_pnns_exploded['pnns_groups_2'].str.replace("'", "", regex=False)

# Supprimer les doublons dans la colonne pour éviter de compter plusieurs fois
df_pnns_exploded = df_pnns_exploded.drop_duplicates(subset='pnns_groups_2')

# Compter les occurrences des groupes
group_counts = df_pnns_exploded['pnns_groups_2'].value_counts()

# Sélectionner les 20 groupes les plus fréquents
top_20_groups = group_counts.head(20)

# Créer un texte géant avec uniquement les 20 groupes les plus fréquents
groups_text = ' '.join([f'{group} ' * freq for group, freq in top_20_groups.items()])

# Générer le WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(groups_text)

# Afficher le WordCloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Pas de bordures ni d'axes
plt.title('WordCloud des 20 Groupes PNNS les Plus Fréquents')
plt.tight_layout(pad=0)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("fichier_concatene_avecpays.csv", low_memory=False)

print(df.head())

In [ ]:
# Identifier les colonnes numériques
numerical_cols = [
    'quantity', 'nutriscore_score', 'energy-kj_100g', 'energy-kcal_100g',
    'fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g',
    'proteins_100g', 'salt_100g', 'fruits-vegetables-nuts-estimate-from-ingredients_100g',
    'nutrition-score-fr_100g'
]

In [ ]:
# Sélectionner les colonnes contenant '_100g' dans leur nom
colonnes_100g = [
    "fat_100g",
    "saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "fruits-vegetables-nuts-estimate-from-ingredients_100g"
]

# Filtrer les lignes où toutes les colonnes '_100g' sont inférieures ou égales à 100
df = df[(df[colonnes_100g] <= 100).all(axis=1)]

# Vérifier le résultat
# Créer un boxplot pour toutes les colonnes numériques
plt.figure(figsize=(12, 8))
sns.boxplot(data=df[numerical_cols], orient='h')  # Orientation horizontale pour plus de lisibilité

# Ajouter des labels et un titre
plt.title('Boxplot des colonnes numériques')
plt.xlabel('Valeurs')
plt.ylabel('Colonnes')

# Afficher le plot
plt.show()
df_archive = df


In [ ]:
# Calculer les valeurs du 99ème percentile pour les colonnes 'energy-kcal_100g' et 'energy-kj_100g'
percentile_99_kcal = df['energy-kcal_100g'].quantile(0.99)
percentile_99_kj = df['energy-kj_100g'].quantile(0.99)

# Filtrer les lignes où 'energy-kcal_100g' et 'energy-kj_100g' sont inférieures ou égales au 99ème percentile
df = df[(df['energy-kcal_100g'] <= percentile_99_kcal) & (df['energy-kj_100g'] <= percentile_99_kj)]

# Vérifier le résultat
print(df.head())

In [ ]:
# Vérifier le résultat
# Créer un boxplot pour toutes les colonnes numériques
plt.figure(figsize=(12, 8))
sns.boxplot(data=df[numerical_cols], orient='h')  # Orientation horizontale pour plus de lisibilité

# Ajouter des labels et un titre
plt.title('Boxplot des colonnes numériques')
plt.xlabel('Valeurs')
plt.ylabel('Colonnes')

# Afficher le plot
plt.show()
df_archive = df

In [ ]:
max_fat_row = df.loc[df['salt_100g'].idxmax()]

# Afficher la ligne avec la valeur maximale de 'fat_100g'
print(max_fat_row)

Boxplot de vérification

In [ ]:

# Convertir 'product_name' et 'brands' en minuscules pour ignorer la casse
df['product_name_lower'] = df['product_name'].str.lower()
df['brands_lower'] = df['brands'].str.lower()

# Supprimer les doublons en utilisant les colonnes en minuscules
df_cleaned = df.drop_duplicates(subset=['product_name_lower', 'brands_lower'], keep='first')

# Supprimer les colonnes temporaires en minuscules
df_cleaned = df_cleaned.drop(columns=['product_name_lower', 'brands_lower'])

print(df_cleaned.head())

On enlève les doublons par marque

In [ ]:
# Vérifier le résultat
# Créer un boxplot pour toutes les colonnes numériques
plt.figure(figsize=(12, 8))
sns.boxplot(data=df_cleaned[numerical_cols], orient='h')  # Orientation horizontale pour plus de lisibilité

# Ajouter des labels et un titre
plt.title('Boxplot des colonnes numériques')
plt.xlabel('Valeurs')
plt.ylabel('Colonnes')

# Afficher le plot
plt.show()

Enregistrement du CSV

In [ ]:
df.to_csv("cleaned_data.csv", index=False)

## Dernière vérification par boxplot 

In [ ]:
# Identifier les colonnes numériques
numerical_cols = [
    'quantity', 'nutriscore_score',
    'fat_100g', 'saturated-fat_100g', 'sugars_100g', 'fiber_100g',
    'proteins_100g', 'salt_100g', 'fruits-vegetables-nuts-estimate-from-ingredients_100g',
    'nutrition-score-fr_100g'
]

In [ ]:
# Créer un boxplot pour toutes les colonnes numériques
plt.figure(figsize=(12, 8))
sns.boxplot(data=df[numerical_cols], orient='h')  # Orientation horizontale pour plus de lisibilité

# Ajouter des labels et un titre
plt.title('Boxplot des colonnes numériques')
plt.xlabel('Valeurs')
plt.ylabel('Colonnes')

# Afficher le plot
plt.show()